# Question 1
Allow the i-item to have both a utility $u i$ and a weight $w i$ Kg. The hiker wishes to choose a subset with the most utility, such that the total weight of the selected goods is less than or equal to 15 kg.

Considering a set of decision binary variables $𝑥_𝑖$ that receive value 1 if the $𝑖-th$ item is selected, and 0 if it is not selected, the resulting mathematical programming formulation is:
$$Maximise\ \sum_{i\in I}u_i.x_i$$
$$s.t.\ \sum_{i\in I}w_i.x_i \le W$$
$$x_i\in\{0,1\}\ \ \ \forall i\in I$$

In [52]:
using JuMP
import Xpress
import GLPK
import Test

In [58]:
model = Model(Xpress.Optimizer)  

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Xpress

In [59]:
utility = [4, 2, 1, 7, 3, 6]
weight = [5, 8, 8, 6, 1, 5]
weight_capacity = 15

15

In [60]:
@variable(model, x[1:6], Bin)

6-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]

In [63]:
#Objective: maximize utility
@objective(model, Max, utility' * x)
#Constraint: can carry all
@constraint(model, weight' * x <= weight_capacity)
#at least 3 items
@constraint(model, sum(x) >= 3)

x[1] + x[2] + x[3] + x[4] + x[5] + x[6] ≥ 3.0

In [62]:
# Solve problem using MIP solver
optimize!(model)
println("The best utility the knapsack can hold is: ", objective_value(model))
println("Solution is:")

for i in 1:6
    print("x[$i] = ", value(x[i]))
    println(", p[$i]/w[$i] = ", utility[i] / weight[i])
end

The best utility the knapsack can hold is: 16.0
Solution is:
x[1] = 0.0, p[1]/w[1] = 0.8
x[2] = 0.0, p[2]/w[2] = 0.25
x[3] = 0.0, p[3]/w[3] = 0.125
x[4] = 1.0, p[4]/w[4] = 1.1666666666666667
x[5] = 1.0, p[5]/w[5] = 3.0
x[6] = 1.0, p[6]/w[6] = 1.2


# Question 2
Consider the following binary integer program.

$$ Maximise\ 12x_1+x_2+10x_3+2x_4$$
$$s.t.$$
$$ 16x_1+2x_2+10x_3+8x_4 \le 18 $$
$$x_i\ is\ binary\ for\ j=1,...,4$$

start by fixing the variable x_1.
Fixing $x_1=0$
$$ Maximise\ x_2+10x_3+2x_4$$
$$s.t.$$
$$ 2x_2+10x_3+8x_4 \le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 2,...,4$$

In [11]:
model = Model(GLPK.Optimizer)  

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [12]:
@variable(model, x1==0)
@variable(model, 0 <= x2 <= 1)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )

x4

In [13]:
#Constraints
@constraint(model, 16x1+2x2+10x3+8x4 <= 18)
# Objective Function:
@objective(model, Max, 12x1+x2+10x3+2x4)
optimize!(model)

In [14]:
if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("the model is infeasible")
else
    println("error!!")
end

x1-x4 value: 0.0, 1.0, 1.0, 0.75
objective value: 12.5


Since this is a max problem, LP gave us a lower bound of 12.5 and this branche have non-integer values. This branch needs to be explored further by fixing other variables. Let's fix x1 = 1.

fixing the variable x_1.
Fixing $x_1=1$
$$ Maximise\ 12+x_2+10x_3+2x_4$$
$$s.t.$$
$$ 2x_2+10x_3+8x_4 \le 2 $$
$$0\le x_i\le 1\ for\ j=\ \ 2,...,4$$

In [15]:
model = Model(GLPK.Optimizer)  

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [16]:
@variable(model, x1==1)
@variable(model, 0 <= x2 <= 1)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )

x4

In [17]:
#Constraints
@constraint(model, 16x1+2x2+10x3+8x4 <= 18)
# Objective Function:
@objective(model, Max, 12x1+x2+10x3+2x4)
optimize!(model)

In [18]:
if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("the model is infeasible")
else
    println("error!!")
end

x1-x4 value: 1.0, 0.0, 0.2, 0.0
objective value: 14.0


Since this branche have non-integer values. This branch still needs to be explored further by fixing other variables. Let's fix x2 = 0.

Fixing $x_1=1,\ x_2=0$
$$ Maximise\ 12+10x_3+2x_4$$
$$s.t.$$
$$ 10x_3+8x_4 \le 2 $$
$$0\le x_i\le 1\ for\ j=\ \ 3,4$$

In [19]:
model = Model(GLPK.Optimizer) 

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [20]:
@variable(model, x1==1)
@variable(model, x2==0)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )

x4

In [21]:
#Constraints
@constraint(model, 16x1+2x2+10x3+8x4 <= 18)
# Objective Function:
@objective(model, Max, 12x1+x2+10x3+2x4)
optimize!(model)

In [22]:
if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("the model is infeasible")
else
    println("error!!")
end

x1-x4 value: 1.0, 0.0, 0.2, 0.0
objective value: 14.0


Since this branche have non-integer values. Let's fix x2 = 1.

fixing the variable x_1.
Fixing $x_1=1$
$$ Maximise\ 12+1+10x_3+2x_4$$
$$s.t.$$
$$ 10x_3+8x_4 \le 0 $$
$$0\le x_i\le 1\ for\ j=\ \ 3,4$$

In [23]:
model = Model(GLPK.Optimizer) 

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [24]:
@variable(model, x1==1)
@variable(model, x2==1)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )

x4

In [25]:
#Constraints
@constraint(model, 16x1+2x2+10x3+8x4 <= 18)
# Objective Function:
@objective(model, Max, 12x1+x2+10x3+2x4)
optimize!(model)

In [26]:
if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("the model is infeasible")
else
    println("error!!")
end

x1-x4 value: 1.0, 1.0, 0.0, 0.0
objective value: 13.0


This branch gave us an integer solution of 13.0. Thus, we can fathom this branch and we don't need to explore this further. 
All other open branches have non-integer values, so even they gives better solution we can not use it.
Thus, we conclude that the optimal objective value is 13.0 and the optimal solution is x1=x2=1 and x3=x4=0.

# Question 3
subject to:
$$Minimise\ \ Z=\sum_{u=1}^{n}x_u$$
$$x_u+x_v\ge 1 \ for \ all \ \ \{u,v\}\in E $$
$$x_u\in\{0,1\} \ \ for \ all\ \ u\in V$$

In [127]:
using JuMP
import GLPK

In [128]:
model = Model(GLPK.Optimizer)  

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [129]:
#the binary variables: 0 = vertices are not selected, 1 = vertices are selected in S1
#connect vertices as the image in the assignment q3
@variable(model, vertice[1:6], Bin)

edges = [vertice[1] + vertice[2], vertice[1] + vertice[3], vertice[2] + vertice[3], vertice[3] + vertice[4], vertice[4] + vertice[5],vertice[4] + vertice[6],vertice[5] + vertice[6]]

7-element Vector{AffExpr}:
 vertice[1] + vertice[2]
 vertice[1] + vertice[3]
 vertice[2] + vertice[3]
 vertice[3] + vertice[4]
 vertice[4] + vertice[5]
 vertice[4] + vertice[6]
 vertice[5] + vertice[6]

In [130]:
#sum of the vertices should greater or equals 1
for i in 1:7
    @constraint(model, edges[i] >=1)
end

In [131]:
#Objective minimise the amount of vertices which connect to all the edges
@objective(model, Min, sum(vertice[i] for i=1:6))
optimize!(model)
println("number of vertices will be selected ", objective_value(model))
for i in 1:6
    println("vertice[$i] = ", value(vertice[i]))
end

number of vertices will be selected 4.0
vertice[1] = 1.0
vertice[2] = 0.0
vertice[3] = 1.0
vertice[4] = 1.0
vertice[5] = 1.0
vertice[6] = 0.0


Consider the linear programming relaxation of ILP formulation 

In [132]:
model = Model(GLPK.Optimizer)  

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [133]:
#the linear progrmming reaxation of ILP means variables can be non-integer
#connect vertices as the image in the assignment q3
@variable(model, 0 <= vertice[1:6] <=1)
edges = [vertice[1] + vertice[2], vertice[1] + vertice[3], vertice[2] + vertice[3], vertice[3] + vertice[4], vertice[4] + vertice[5],vertice[4] + vertice[6],vertice[5] + vertice[6]]

7-element Vector{AffExpr}:
 vertice[1] + vertice[2]
 vertice[1] + vertice[3]
 vertice[2] + vertice[3]
 vertice[3] + vertice[4]
 vertice[4] + vertice[5]
 vertice[4] + vertice[6]
 vertice[5] + vertice[6]

In [134]:
#sum of the vertices should greater or equals 1
for i in 1:7
    @constraint(model, edges[i] >=1)
end

In [135]:
#Objective minimise the amount of vertices which connect to all the edges
@objective(model, Min, sum(vertice[i] for i=1:6))
optimize!(model)
println("number of vertices will be selected ", objective_value(model))
for i in 1:6
    println("vertice[$i] = ", value(vertice[i]))
end

number of vertices will be selected 3.0
vertice[1] = 0.5
vertice[2] = 0.5
vertice[3] = 0.5
vertice[4] = 0.5
vertice[5] = 0.5
vertice[6] = 0.5


If S3 is set of vertices for which the corresponding variable in S2,we can then round: 
$$x_1 = 1 \ \ since \ \ vertice[1] = 0.5$$
$$x_2 = 1 \ \ since \ \ vertice[2] = 0.5$$
$$x_3 = 1 \ \ since \ \ vertice[3] = 0.5$$
$$x_4 = 1 \ \ since \ \ vertice[4] = 0.5$$
$$x_5 = 1 \ \ since \ \ vertice[5] = 0.5$$
$$x_6 = 1 \ \ since \ \ vertice[6] = 0.5$$
As the result we can discover that the objective value above is 6 which means that all the vertices are selected, and it is almost the double size of the optimal sloution we had before. So we can come out with conclusion that this solution will not be worked.